In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import gc
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_pickle('/kaggle/input/riiid-train-data-multiple-formats/riiid_train.pkl.gzip')
questions = pd.read_csv('/kaggle/input/riiid-test-answer-prediction/questions.csv', dtype={'question_id':np.int16, 'bundle_id': np.int16, 'correct_answer': np.int8, 'part': np.int8, 'tags': np.str })
lectures = pd.read_csv('/kaggle/input/riiid-test-answer-prediction/lectures.csv')

In [ ]:
user_stats = train[train['content_type_id'] == False].groupby(['user_id']).answered_correctly.agg(['count', 'sum'])

In [ ]:
user_stats_dict = user_stats.to_dict()
keys_list = set(user_stats_dict['count'].keys())

In [ ]:
def update_user_stats(df):
    user_stats_temp = df[df['content_type_id'] == False].groupby(['user_id']).answered_correctly.agg(['count', 'sum'])
    user_stats_temp_dict = user_stats_temp.to_dict()
    user_stats_temp_dict_cnt = user_stats_temp_dict['count']
    user_stats_temp_dict_sum = user_stats_temp_dict['sum']
    for key, val in user_stats_temp_dict_cnt.items():
        keys_list.add(key)
        user_stats_dict['count'][key] = user_stats_temp_dict_cnt.get(key,0)+val
    for key, val in user_stats_temp_dict_sum.items():
        user_stats_dict['sum'][key] = user_stats_temp_dict_sum.get(key,0)+val

In [ ]:
def inference():
    import riiideducation
    import time
    # You can only call make_env() once, so don't lose it!
    env = riiideducation.make_env()
    iter_test = env.iter_test()
    previous_test_df = None
    i = 0
    print("Starting inference...")
    for (test_df, sample_prediction_df) in iter_test:
        i+=1
        curr_time = time.time()
        if previous_test_df is not None:
            previous_test_df['answered_correctly'] = eval(test_df["prior_group_answers_correct"].iloc[0])
            update_user_stats(previous_test_df)
        previous_test_df = test_df.copy()
        test_df = test_df[test_df['content_type_id'] == 0].reset_index(drop = True)

        pred_list = []
        for idx, row in test_df.iterrows():
            if row['user_id'] not in keys_list:
                pred_list.append(0.5)
            else:
                pred_list.append(user_stats_dict['sum'][row['user_id']] / user_stats_dict['count'][row['user_id']])
        test_df = test_df.assign(answered_correctly = pred_list) 
        env.predict(test_df[['row_id', 'answered_correctly']])
        print("Iteration took: ", time.time()-curr_time, " seconds")

In [ ]:
inference()